In [1]:
cd drive/MyDrive/clean/

/content/drive/MyDrive/clean


In [ ]:
cd clean/darknet/

/content/drive/My Drive/clean/darknet


In [ ]:
cd ..

/content/drive/My Drive


In [5]:
cd darknet/

/content/drive/My Drive/clean/darknet


In [ ]:
cd /content/drive/MyDrive/aze/videos

/content/drive/MyDrive/aze/videos


In [ ]:
ls

IMG_0143.MOV  IMG_0146.MOV       trimed_video.mp4
IMG_0145.MOV  trimed_video2.mp4  videoplayback.mp4


In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip("videoplayback.mp4", 180, 240, targetname="trimed_video2.mp4")


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i videoplayback.mp4 -ss 180.00 -t 60.00 -vcodec copy -acodec copy trimed_video2.mp4
... command successful.


In [2]:
# Libraries Import
import numpy as np
import cv2
import time
import pandas as pd
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.models import load_model

In [3]:
# Reading video File
video = cv2.VideoCapture('night_video.mp4')

In [4]:
# Writer for saving video
writer = None

# Variables for frame dimensions
h, w = None, None

In [6]:
# Yolo Class names
with open('cfg/classes.names') as f:
    labels = [line.strip() for line in f]
    print(labels)

# CNN Class Names
signs = pd.read_csv('label_names.csv')

# CNN Model
model = load_model('model.h5')
print('signs')

# Loading Yolo Weights
network = cv2.dnn.readNetFromDarknet('cfg/yolov4-custom_test.cfg','yolov4-custom_last_TEST.weights')

['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-Red', 'trafficLight-Yellow', 'truck', 'traffic_sign']
signs


In [7]:
# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

# Getting Unconnected Layers YOLO
layers_names_output = [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

In [8]:
# Minimum Probability to eliminate weak predictions
probability_minimum = 0.5

# Threshold for filtering weak bounding boxes with maximum supression
threshold = 0.3

# Random Colors for each yolo Class
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')


# Frame Count
f = 0

# Time Spent
t = 0

In [9]:
# Function for Grayscaling the image
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

# Function for equalizing image using Histogram Equalizer
def equalize(img):
    img = cv2.equalizeHist(img)
    return img

# Preprocessing function that uses Grayscaling, Equalizing and Normalizing by division to 255
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

In [10]:
# Defining loop for catching frames
while True:
    # Capturing frame-by-frame
    ret, frame = video.read()

    # If no frame then break loop
    if not ret:
        break

    # Getting dimensions of the frame
    if w is None or h is None:
        # Slicing from tuple only first two elements
        h, w = frame.shape[:2]

    # Getting blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)

    # Forward pass using blob
    network.setInput(blob) 
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters for frames and total time
    f += 1
    t += end - start

    # Showing spent time for single current frame
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))


    # List variables for detected objects
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 81 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]

            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:
                # Scaling bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Coordinates from YOLO
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min,
                                       int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)


    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold)


    # Checking if there is any object
    # after non-maximum suppression
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # If detected object is a traffic sign
            if (class_numbers[i] == 8):
                # Cut fragment with Traffic Sign
                c_ts = frame[y_min:y_min + int(box_height), x_min:x_min + int(box_width), :]

                if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                    pass
                else:

                    # Preprocessing frame
                    img = np.asarray(c_ts)
                    img = cv2.resize(img, (32, 32))
                    img = preprocessing(img)
                    img = img.reshape(1, 32, 32, 1)

                    # Feeding to the Keras CNN model to get predicted label among 81 classes
                    scores = model.predict(img)

                    # Scores is given for image with 81 numbers of predictions for each class
                    prediction = np.argmax(scores)

                    # Probability of the Classification
                    probabilityValue = np.amax(scores)

                    # Colour for current bounding box
                    colour_box_current = colours[class_numbers[i]].tolist()

                    # Drawing bounding box on the original current frame
                    cv2.rectangle(frame, (x_min, y_min),
                                  (x_min + box_width, y_min + box_height),
                                  colour_box_current, 2)

                    # Preparing text with label and confidence for current bounding box
                    text_box_current = '{}: {:.4f}'.format(signs['SignName'][prediction],
                                                           probabilityValue)

                    # Putting text with label and confidence on the original image
                    cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)
            else:
                # Colour for current bounding box
                colour_box_current = colours[class_numbers[i]].tolist()

                # Drawing bounding box on the original current frame
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                       confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    # Initializing writer
    if writer is None:
        # Constructing code of the codec
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('night_demo.mp4', fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    # Write processed current frame to the file
    writer.write(frame)

Frame number 1 took 11.85101 seconds
Frame number 2 took 1.49947 seconds
Frame number 3 took 1.34503 seconds
Frame number 4 took 1.34430 seconds
Frame number 5 took 1.34749 seconds
Frame number 6 took 1.34475 seconds
Frame number 7 took 1.34754 seconds
Frame number 8 took 1.34697 seconds
Frame number 9 took 1.34146 seconds
Frame number 10 took 1.34439 seconds
Frame number 11 took 1.34321 seconds
Frame number 12 took 1.34055 seconds
Frame number 13 took 1.34758 seconds
Frame number 14 took 1.33947 seconds
Frame number 15 took 1.33726 seconds
Frame number 16 took 1.34791 seconds
Frame number 17 took 1.35124 seconds
Frame number 18 took 1.34158 seconds
Frame number 19 took 1.33873 seconds
Frame number 20 took 1.34357 seconds
Frame number 21 took 1.34077 seconds
Frame number 22 took 1.34365 seconds
Frame number 23 took 1.34203 seconds
Frame number 24 took 1.33987 seconds
Frame number 25 took 1.33704 seconds
Frame number 26 took 1.33740 seconds
Frame number 27 took 1.34215 seconds
Frame num

In [11]:
# Printing final results
print()
print('Total number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print('FPS:', round((f / t), 1))


Total number of frames 513
Total amount of time 696.01054 seconds
FPS: 0.7


In [12]:
# Releasing video reader and writer
video.release()
writer.release()

In [ ]:
cv2.imshow()

/content/drive/My Drive


In [1]:
!pip install opencv-contrib-python==3.4.13.47 --force-reinstall

     |████████████████████████████████| 53.7MB 53kB/s 
     |████████████████████████████████| 15.3MB 183kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
